In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
df = pd.read_csv('Phishing_Clened_EDA_data.csv')

In [4]:
df.head()

,Total-character,Uppercase_letter,Lowercase_letter,Total_numbers,Total_symbol,Total_slesh,Total_dots,Total_desh,Label
0,225,2,133,58,32,10,6,4,0
1,81,0,65,1,15,4,5,2,0
2,177,0,111,47,19,11,7,1,0
3,60,0,52,0,8,2,6,0,0
4,116,12,70,21,13,10,1,1,0


In [5]:
# convert depedent and indepedent feature
X = df.drop(['Label'],axis=1)
y = df['Label']

In [6]:
X.head()

,Total-character,Uppercase_letter,Lowercase_letter,Total_numbers,Total_symbol,Total_slesh,Total_dots,Total_desh
0,225,2,133,58,32,10,6,4
1,81,0,65,1,15,4,5,2
2,177,0,111,47,19,11,7,1
3,60,0,52,0,8,2,6,0
4,116,12,70,21,13,10,1,1


In [8]:
y.value_counts()

Label
1    392924
0    156422
Name: count, dtype: int64

In [9]:
# convert into X_train and test
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42)

In [11]:
X_train.shape,X_test.shape

((384542, 8), (164804, 8))

In [13]:
import pickle
with open('../scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [14]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
X_train_scaled

array([[-0.73790045, -0.19418911, -0.87292671, ..., -0.90626831,
        -0.70308642, -0.05274225],
       [ 0.47901566,  0.08031588,  0.60104817, ...,  0.97094411,
        -0.70308642, -0.05274225],
       [-0.37733272,  0.17181755, -0.47455512, ..., -0.28053084,
        -0.04057017, -0.46415897],
       ...,
       [-0.94071981, -0.19418911, -1.19162398, ..., -0.90626831,
        -0.70308642, -0.46415897],
       [-0.3998682 , -0.19418911, -0.35504365, ..., -0.90626831,
         1.28446234, -0.46415897],
       [-0.49001013, -0.19418911, -0.47455512, ...,  0.34520664,
        -0.70308642, -0.46415897]])

## DL Model Training 


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [18]:
(X_train_scaled.shape[1],)

(8,)

In [21]:
model  = Sequential()
model.add(Dense(units=64,activation='relu',input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(units=32,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

E:\PhishNet-Phishing-URL-Detection-System\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [24]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [25]:
early_stop_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [26]:
history = model.fit(
    X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),
    epochs=100,
    callbacks=[early_stop_callback]
)

Epoch 1/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 67s 5ms/step - accuracy: 0.8196 - loss: 0.4058 - val_accuracy: 0.8353 - val_loss: 0.3787
Epoch 2/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - accuracy: 0.8335 - loss: 0.3743 - val_accuracy: 0.8366 - val_loss: 0.3692
Epoch 3/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 81s 5ms/step - accuracy: 0.8376 - loss: 0.3666 - val_accuracy: 0.8380 - val_loss: 0.3608
Epoch 4/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 57s 5ms/step - accuracy: 0.8374 - loss: 0.3651 - val_accuracy: 0.8388 - val_loss: 0.3632
Epoch 5/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - accuracy: 0.8379 - loss: 0.3644 - val_accuracy: 0.8401 - val_loss: 0.3586
Epoch 6/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - accuracy: 0.8394 - loss: 0.3591 - val_accuracy: 0.8381 - val_loss: 0.3635
Epoch 7/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - accuracy: 0.8380 - loss: 0.3619 - val_accuracy: 0.8394 - val_loss: 0.3630
Epoch 8/100
12017/12017 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - ac

In [29]:
# load model
y_pred_prob = model.predict(X_test_scaled)

5151/5151 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step


In [39]:
y_pred_prob[0]

array([0.9774297], dtype=float32)

In [40]:
y_pred = []
for i in range(len(y_pred_prob)):
    if y_pred_prob[i][0] > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [41]:
y_pred 

[1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [42]:
# calculate the score and confusion metrix
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_auc_score

In [43]:
score = accuracy_score(y_test,y_pred)
report = classification_report(y_test,y_pred)
matrix = confusion_matrix(y_test,y_pred)
roc_score = roc_auc_score(y_test,y_pred)

In [44]:
print(f'accuracy_score:{score}')
print('Classification_report:\n {}'.format(report))
print(f'confusion_matrix:\n {matrix}')
print(f'roc_auc_score:{roc_auc_score}')

accuracy_score:0.8419880585422684
Classification_report:
               precision    recall  f1-score   support

           0       0.85      0.54      0.66     46750
           1       0.84      0.96      0.90    118054

    accuracy                           0.84    164804
   macro avg       0.85      0.75      0.78    164804
weighted avg       0.84      0.84      0.83    164804

confusion_matrix:
 [[ 25152  21598]
 [  4443 113611]]
roc_auc_score:<function roc_auc_score at 0x000002BBF61CB380>


In [50]:
# save deep learning model
from tensorflow.keras.models import save_model

In [51]:
save_model(model,'../DL_model.h5')